# 学習率の検討・学習率annealing・adabound

## 適切な Learning Rate 探索
毎iterationごとに Adam の alpha の値を徐々に大きくしていって、loss, accuracyの変化を観察する  
loss の低下が止まる直前の値が最も良さげな値といえる

In [ ]:
# learning rate の scheduler

import chainer

class LRSchedule(chainer.training.Extension):
    def __init__(self, attr, log_range=[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e-0]):
        self._attr = attr
        self._t = 0
        self._log_range = log_range
        self._lr_gen = self._lr_generator()

    def __call__(self, trainer):
        optimizer = trainer.updater.get_optimizer('main')
        setattr(optimizer, self._attr, next(self._lr_gen))
        
    def _lr_generator(self):
        for val in self._log_range:
            for i in range(1, 100):
                yield val * (1 + i / 10)

In [ ]:
import chainer
from chainer.dataset import convert
from chainer.training import extensions
import pandas as pd

import melanoma

device = 0
batch_size = 32
width = 336
height = 224
n_classes = 2
iteration = 99 * 6
output_dir = melanoma.constants.PROJECT_ROOT / "results" / "lr_inspection"

model = melanoma.models.EfficientNet()
loss_func = melanoma.models.loss.SigmoidLoss(model)
optimizer = chainer.optimizers.Adam(alpha=1e-5, beta1=0.0, beta2=0.0)

if device >= 0:
    model.to_gpu(device)

optimizer.setup(model)

df = pd.read_csv(melanoma.dataset.DATASET_ROOT / "train.csv")
ds = melanoma.dataset.Dataset(df, n_classes, (width, height), preprocess=melanoma.imageproc.standard_augmentation)
itr = chainer.iterators.MultiprocessIterator(ds, batch_size, repeat=True)
updater = chainer.training.StandardUpdater(
    itr, optimizer, converter=convert.concat_examples, loss_func=loss_func, device=device
)
trainer = chainer.training.Trainer(updater, (iteration, 'iteration'), out=output_dir)
trainer.extend(extensions.observe_lr(), trigger=(1, 'iteration'))
trainer.extend(extensions.PrintReport(['iteration', 'lr', 'main/loss', 'main/accuracy']))
trainer.extend(extensions.LogReport(trigger=(1, 'iteration')))
melanoma.utility.start_pdb(
    lambda: trainer.extend(LRSchedule("alpha"))
)

In [ ]:
melanoma.utility.start_pdb(
    lambda: trainer.run()
)


In [ ]:
import json

with open(output_dir / "log") as f:
    json_dict = json.load(f)
    
lrs = [j["lr"] for j in json_dict]
losses = [j["main/loss"] for j in json_dict]
accs = [j["main/accuracy"] for j in json_dict]    

## 結果
alpha = 5e-5 あたりが一番良さそう

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(15, 10))
axes[0].plot(lrs[:200], losses[:200])
axes[0].set_xscale("log")
axes[0].set_yscale("log")
axes[1].plot(lrs[:200], accs[:200])
axes[1].set_xscale("log")